In [3]:
import os
import fitz  # PyMuPDF
import uuid
import base64
import openai
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
# Load environment variables
load_dotenv()
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_CLUSTER_URL = os.getenv("QDRANT_CLUSTER_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [4]:
# Configure OpenAI
openai.api_key = OPENAI_API_KEY
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [5]:
import os
import fitz  # PyMuPDF
import uuid
import base64
from dotenv import load_dotenv
from openai import OpenAI
from PIL import Image
import pytesseract
import io

# Load API keys
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

def image_to_markdown(image_data):
    # Add padding to image to prevent text cutoff at edges
    image = Image.open(io.BytesIO(image_data))
    
    # Add padding on all sides (top, bottom, left, right)
    padding = 30
    padded_image = Image.new('RGB', 
                            (image.width + 2*padding, image.height + 2*padding), 
                            (255, 255, 255))
    padded_image.paste(image, (padding, padding))
    
    # Convert back to bytes
    buffer = io.BytesIO()
    padded_image.save(buffer, format='PNG')
    padded_image_data = buffer.getvalue()
    
    base64_image = base64.b64encode(padded_image_data).decode('utf-8')

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": (
                            "Extract ALL visible text content from this image without skipping or summarizing ANYTHING. "
                            "This is critical: capture EVERY single word, number, symbol, and line visible in the image, including: "
                            "- Headers, subheaders, and titles "
                            "- Body text and paragraphs "
                            "- Page numbers, footers, and copyright notices "
                            "- Table content and formatting "
                            "- Lists, bullet points, and numbered items "
                            "- Any text at the very bottom, top, or edges of the page "
                            "- Small text, footnotes, and annotations "
                            "Format everything in proper Markdown while preserving the exact structure and layout. "
                            "Do NOT omit any content, especially at page boundaries. Return ONLY the Markdown content."
                        )
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        max_tokens=4096
    )

    return response.choices[0].message.content

def extract_text_with_ocr(image_data):
    image = Image.open(io.BytesIO(image_data))
    
    # Add OCR configuration for better text capture
    custom_config = r'--oem 3 --psm 6 -c preserve_interword_spaces=1'
    return pytesseract.image_to_string(
        image,
        config=custom_config,
        lang='eng'
    )

# Load PDF and convert pages to markdown with OCR fallback
pdf_path = "./Cropped_Admin_guide_1.pdf"
doc = fitz.open(pdf_path)
markdown_docs = []

for i, page in enumerate(doc):
    print(f"Processing page {i+1}/{len(doc)}...")
    
    # Render page at higher resolution
    pix = page.get_pixmap(matrix=fitz.Matrix(3.0, 3.0))
    image_data = pix.tobytes("png")

    # Get GPT-4 Vision markdown
    print(f"  Getting GPT-4o markdown for page {i+1}...")
    markdown_content = image_to_markdown(image_data)

    # OCR fallback with improved settings
    print(f"  Running OCR for page {i+1}...")
    ocr_text = extract_text_with_ocr(image_data)

    # Comprehensive OCR fallback - check for missing content
    ocr_lines = [line.strip() for line in ocr_text.strip().splitlines() if line.strip()]
    
    # Check last 5 lines and first 3 lines from OCR for missing content
    lines_to_check = []
    if len(ocr_lines) >= 5:
        lines_to_check.extend(ocr_lines[:3])  # First 3 lines
        lines_to_check.extend(ocr_lines[-5:])  # Last 5 lines
    else:
        lines_to_check = ocr_lines
    
    # Remove duplicates while preserving order
    lines_to_check = list(dict.fromkeys(lines_to_check))
    
    # Append missing lines
    missing_lines = []
    for line in lines_to_check:
        if line and len(line) > 3:  # Only check substantial lines
            # Check if line content is missing (not just exact match)
            line_words = line.split()
            if len(line_words) >= 2:  # Only check lines with multiple words
                found = any(all(word.lower() in markdown_content.lower() for word in line_words[:3]) 
                           for word in line_words)
                if not found:
                    missing_lines.append(line)
    
    # Add missing content
    if missing_lines:
        print(f"  Found {len(missing_lines)} missing lines for page {i+1}")
        markdown_content += "\n\n<!-- OCR Fallback - Missing Content -->"
        for line in missing_lines:
            markdown_content += f"\n{line}"
    else:
        print(f"  No missing content detected for page {i+1}")
    
    # Add page boundary marker
    markdown_content += f"\n\n<!-- End of Page {i+1} -->"

    # Extract first Markdown header as section heading
    heading = ""
    for line in markdown_content.split('\n'):
        if line.strip().startswith('#'):
            heading = line.strip().lstrip('#').strip()
            break

    markdown_docs.append({
        "content": markdown_content,
        "ocr_text": ocr_text,  # Full OCR content for debugging
        "metadata": {
            "source": "Polarion Admin Guide",
            "page_number": i + 1,
            "page_width": page.rect.width,
            "page_height": page.rect.height,
            "heading": heading
        }
    })


Processing page 1/41...
  Getting GPT-4o markdown for page 1...
  Running OCR for page 1...
  No missing content detected for page 1
Processing page 2/41...
  Getting GPT-4o markdown for page 2...
  Running OCR for page 2...
  Found 1 missing lines for page 2
Processing page 3/41...
  Getting GPT-4o markdown for page 3...
  Running OCR for page 3...
  No missing content detected for page 3
Processing page 4/41...
  Getting GPT-4o markdown for page 4...
  Running OCR for page 4...
  Found 4 missing lines for page 4
Processing page 5/41...
  Getting GPT-4o markdown for page 5...
  Running OCR for page 5...
  Found 2 missing lines for page 5
Processing page 6/41...
  Getting GPT-4o markdown for page 6...
  Running OCR for page 6...
  Found 2 missing lines for page 6
Processing page 7/41...
  Getting GPT-4o markdown for page 7...
  Running OCR for page 7...
  Found 2 missing lines for page 7
Processing page 8/41...
  Getting GPT-4o markdown for page 8...
  Running OCR for page 8...
  Found

In [6]:
# Print the content of the first markdown page
print("First page content:")
print(markdown_docs[24]["content"])
print("\nMetadata:")
print(markdown_docs[24]["metadata"])


First page content:
```markdown
## Understand Plan actions

The sum of the remaining estimates of all planned items, or the sum of **Story Points** on unresolved items.

° **Done:**

The sum of the **Time Spent** on all planned items. (Sometimes days spent before the plan was started are subtracted.)

Or;

The sum of **Story Points** on resolved items.

- **Default Estimate:** Specify a value to use for **Work Items' Initial Estimate** field. Use as the default for any **Work Items** in the Plan that have no value set for their Initial Estimate field.

- **Previous Time Spent:** Stores the sum of the **Time Spend** field for a Plan that contains **Work Items** which has some time already spent. That is, the items contain a value in their **Time Spend** field. This property is updated automatically by the system and you should not normally need to edit it. You can find detail about the calculation in **Previous Time Spent Property**.

The Plan properties page also provides a field for a

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Use original Markdown without cleaning
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

docs_to_embed = []
for doc in markdown_docs:
    chunks = splitter.create_documents(
        texts=[doc["content"]],        # Use raw markdown as-is
        metadatas=[doc["metadata"]]    # Preserve metadata
    )
    docs_to_embed.extend(chunks)

In [ ]:
# from langchain_experimental.text_splitter import SemanticChunker
# from langchain.docstore.document import Document
# from langchain_openai import OpenAIEmbeddings

# # Initialize semantic chunker with embeddings
# semantic_chunker = SemanticChunker(
#     embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),
#     breakpoint_threshold_type="percentile",  # Options: "percentile", "standard_deviation", "interquartile"
#     breakpoint_threshold_amount=95,  # Threshold for semantic breaks (95th percentile)
#     number_of_chunks=None,  # Let it determine optimal number of chunks
#     sentence_split_regex=r'(?<=[.!?])\s+',  # Split on sentence boundaries
# )

# docs_to_embed = []
# for doc in markdown_docs:
#     print(f"Creating semantic chunks for page {doc['metadata']['page_number']}...")
    
#     # Create semantic chunks
#     chunks = semantic_chunker.create_documents(
#         texts=[doc["content"]],        # Use raw markdown as-is
#         metadatas=[doc["metadata"]]    # Preserve metadata
#     )
    
#     # Add chunk index to metadata for better tracking
#     for idx, chunk in enumerate(chunks):
#         chunk.metadata["chunk_index"] = idx
#         chunk.metadata["total_chunks_in_page"] = len(chunks)
    
#     docs_to_embed.extend(chunks)
#     print(f"  Created {len(chunks)} semantic chunks for page {doc['metadata']['page_number']}")

# print(f"\nTotal semantic chunks created: {len(docs_to_embed)}")

Creating semantic chunks for page 1...
  Created 1 semantic chunks for page 1
Creating semantic chunks for page 2...
  Created 3 semantic chunks for page 2
Creating semantic chunks for page 3...
  Created 2 semantic chunks for page 3
Creating semantic chunks for page 4...
  Created 2 semantic chunks for page 4
Creating semantic chunks for page 5...
  Created 2 semantic chunks for page 5
Creating semantic chunks for page 6...
  Created 3 semantic chunks for page 6
Creating semantic chunks for page 7...
  Created 3 semantic chunks for page 7
Creating semantic chunks for page 8...
  Created 3 semantic chunks for page 8
Creating semantic chunks for page 9...
  Created 2 semantic chunks for page 9
Creating semantic chunks for page 10...
  Created 2 semantic chunks for page 10
Creating semantic chunks for page 11...
  Created 3 semantic chunks for page 11
Creating semantic chunks for page 12...
  Created 2 semantic chunks for page 12
Creating semantic chunks for page 13...
  Created 2 semant

In [ ]:
# from langchain_experimental.text_splitter import SemanticChunker
# from langchain.docstore.document import Document
# from langchain_openai import OpenAIEmbeddings

# # Initialize semantic chunker with embeddings
# semantic_chunker = SemanticChunker(
#     embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),
#     breakpoint_threshold_type="percentile",  # Options: "percentile", "standard_deviation", "interquartile"
#     breakpoint_threshold_amount=95,  # Threshold for semantic breaks (95th percentile)
#     number_of_chunks=None,  # Let it determine optimal number of chunks
#     sentence_split_regex=r'(?<=[.!?])\s+',  # Split on sentence boundaries
# )

# docs_to_embed = []
# for doc in markdown_docs:
#     print(f"Creating semantic chunks for page {doc['metadata']['page_number']}...")
    
#     # Create semantic chunks
#     chunks = semantic_chunker.create_documents(
#         texts=[doc["content"]],        # Use raw markdown as-is
#         metadatas=[doc["metadata"]]    # Preserve metadata
#     )
    
#     # Add chunk index to metadata for better tracking
#     for idx, chunk in enumerate(chunks):
#         chunk.metadata["chunk_index"] = idx
#         chunk.metadata["total_chunks_in_page"] = len(chunks)
    
#     docs_to_embed.extend(chunks)
#     print(f"  Created {len(chunks)} semantic chunks for page {doc['metadata']['page_number']}")

# print(f"\nTotal semantic chunks created: {len(docs_to_embed)}")

Creating semantic chunks for page 1...
  Created 1 semantic chunks for page 1
Creating semantic chunks for page 2...
  Created 3 semantic chunks for page 2
Creating semantic chunks for page 3...
  Created 2 semantic chunks for page 3
Creating semantic chunks for page 4...
  Created 2 semantic chunks for page 4
Creating semantic chunks for page 5...
  Created 2 semantic chunks for page 5
Creating semantic chunks for page 6...
  Created 3 semantic chunks for page 6
Creating semantic chunks for page 7...
  Created 3 semantic chunks for page 7
Creating semantic chunks for page 8...
  Created 3 semantic chunks for page 8
Creating semantic chunks for page 9...
  Created 2 semantic chunks for page 9
Creating semantic chunks for page 10...
  Created 2 semantic chunks for page 10
Creating semantic chunks for page 11...
  Created 3 semantic chunks for page 11
Creating semantic chunks for page 12...
  Created 2 semantic chunks for page 12
Creating semantic chunks for page 13...
  Created 2 semant

In [8]:
# Print first few documents from docs_to_embed
print("Number of chunks:", len(docs_to_embed))
print("\nFirst 3 chunks:")
for i, doc in enumerate(docs_to_embed[:3]):
    print(f"\nChunk {i+1}:")
    print("Content:", doc.page_content[:200] + "...")  # Show first 200 chars
    print("Metadata:", doc.metadata)


Number of chunks: 68

First 3 chunks:

Chunk 1:
Content: ```markdown
[Access Project Work Items](#)

---

2-154

Software Version Polarion 2410     

Administrator and User Help  
Unpublished work. © 2024 Siemens
```

<!-- End of Page 1 -->...
Metadata: {'source': 'Polarion Admin Guide', 'page_number': 1, 'page_width': 612.0, 'page_height': 792.0, 'heading': ''}

Chunk 2:
Content: ```markdown
Home

3. User guide

Home

About Home pages

Every project in the Polarion portal has a Home page. The default content is provided by the project template. **Home** page content can be mod...
Metadata: {'source': 'Polarion Admin Guide', 'page_number': 2, 'page_width': 612.0, 'page_height': 792.0, 'heading': ''}

Chunk 3:
Content: ---

Administrator and User Help  
Unpublished work. © 2024 Siemens

3-1  
Software Version Polarion 2410
```

<!-- OCR Fallback - Missing Content -->
¢ Default content of a Project home page is provi...
Metadata: {'source': 'Polarion Admin Guide', 'page_number': 2, 'page

In [14]:
qdrant_client = QdrantClient(url=QDRANT_CLUSTER_URL, api_key=QDRANT_API_KEY)

In [9]:
qdrant_client = QdrantClient(url=QDRANT_CLUSTER_URL, api_key=QDRANT_API_KEY)

collection_name = "polarion_admin_guide_chunks_jun2"
if collection_name not in [col.name for col in qdrant_client.get_collections().collections]:
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
    )

points = []
for chunk in docs_to_embed:
    vector = embedding_model.embed_query(chunk.page_content)
    payload = chunk.metadata.copy()
    payload["page_content"] = chunk.page_content
    points.append(models.PointStruct(
        id=str(uuid.uuid4()),
        payload=payload,
        vector=vector
    ))

qdrant_client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [10]:
from langchain.vectorstores import Qdrant
qdrant_vectorstore = Qdrant(
        client=qdrant_client,
        embeddings=embedding_model,
        collection_name="polarion_admin_guide_chunks_jun2"
    )

/tmp/ipykernel_60952/3691454962.py:2: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  qdrant_vectorstore = Qdrant(


In [16]:
retriever = qdrant_vectorstore.as_retriever(search_kwargs={"k": 10})

In [17]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI chat model
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.2,
    streaming=True,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)



In [93]:
#llm.invoke("How to convert home page from classic wiki format?")

In [18]:
from langchain_core.prompts import ChatPromptTemplate
# Define the RAG prompt template
RAG_PROMPT = """
You are **Polarion AI Assistant**, a specialist in Polarion ALM
configuration, customization, and administration.

**Ground rules**

1. Use **ONLY** the information in the CONTEXT.  
2. If the CONTEXT does **not** contain the answer, say  
   “I’m not sure from the provided documentation.”  
3. Never invent URLs, file names, or steps that are not in the CONTEXT.  
4. Cite the **page_number** (from metadata) or any other supplied locator 
   when you reference a fact - e.g. “(p. 23)”.

---

**CONTEXT**
{context}

---

**USER QUESTION**
{question}

---

**Respond in this format**

**Answer**  
A direct, concise answer to the question.

**Supporting details**  
• Bullet-point evidence or step-by-step instructions, each followed by a citation.

Example:
• Go to *Administration ▶ Home Page* (p. 42)  
• Switch the *Content type* field from “Classic Wiki” to “Rich Text” (p. 43)

If no answer is possible:
“*I’m not sure from the provided documentation.*”
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [19]:
# Create the RAG chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

rag_chain =  (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [21]:
rag_chain.invoke({"question": "How to convert home page from classic wiki format?"})["context"]

[Document(metadata={'_id': '91049e54-ed62-47c4-a504-92321e4c45cf', '_collection_name': 'polarion_admin_guide_semantic_chunks'}, page_content="```markdown\n# Convert Home page from the Classic Wiki format\n\nFrom version 2016-SR1, new users' personal home pages (and other default pages such as Repository and space Home pages) are created as LiveReport type Pages. This technology replaces the older Classic Wiki technology for information and report pages. LiveReport type Pages are easier for nontechnical users; thanks to construction using visually configurable Widgets rather than mark-up language and code. If you have an existing My Polarion (or other default) page created with a Polarion version prior to 2016-SR1, you can easily convert your existing page to use the newer and easier Pages technology. ### Identify the Page's format\n\n#### Procedure\n\n1. Log on to your Polarion portal, and in Navigation click **My Polarion** (or select any space's **Home** node). 2."),
 Document(metada

In [20]:
rag_chain.invoke({"question": "List all the Plan basic properties?"})["context"]

[Document(metadata={'_id': '97f465c7-e015-4290-bab1-8547dac18c57', '_collection_name': 'polarion_admin_guide_chunks_jun2'}, page_content='### To access and edit Plan properties:\n\n1. Click the **Properties** button on the Plan detail pane toolbar (or Plan page toolbar if you are not viewing the Plan in the table).\n\n2. Click the **Edit** button to put the Plan Properties form into edit mode. Note that this may not always be necessary, as many of the property fields can be edited in-place. Such fields provide visual feedback when you hover over them.\n\n### Basic properties\n\nKey properties to focus on include:\n\n---\n\n3-20  \nSoftware Version Polarion 2410  \nAdministrator and User Help  \nUnpublished work. © 2024 Siemens  \n```\n\n\n<!-- End of Page 21 -->'),
 Document(metadata={'_id': '3464cc97-6200-42f1-937b-85bd24c74d59', '_collection_name': 'polarion_admin_guide_chunks_jun2'}, page_content="- **Parent**: The ID of the Plan that is the immediate parent of the current Plan, if 

In [22]:
rag_chain.invoke({"question": "How to convert home page from classic wiki format?"})["response"].content

"**Answer**  \nTo convert your home page from Classic Wiki format to LiveReport format, follow these steps.\n\n**Supporting details**  \n• Log on to your Polarion portal, and in Navigation click **My Polarion** (or select any space's **Home** node) (p. 3).  \n• At the top of the page, click **Expand Tools**, and then click **Edit**. If you see the **Widgets** sidebar on the right, your page is already using the newer Pages technology and does not need conversion. If you see the **Wiki Syntax Help** sidebar, your page is using the Classic Wiki technology (p. 3).  \n• Access your My Polarion page and click **Expand Tools** (p. 3).  \n• Click the option to **Switch To LiveReport Page** in the menu (p. 3).  \n• In the next dialog box, click **Switch** to convert the page (p. 3).  \n• Customize the new LiveReport page using the source of the former Classic Wiki report as a guide (p. 3)."

In [ ]:
rag_chain.invoke({"question": "How to access user account page?"})["response"].content

NameError: name 'rag_chain' is not defined

In [27]:
rag_chain.invoke({"question": "List the plan properties?"})["context"]

[Document(metadata={'_id': '1182c866-bc63-4d68-9425-30f4d458566b', '_collection_name': 'polarion_admin_guide_semantic_chunks'}, page_content='Generally, **Plan** properties should be reviewed and set before **Work Items** are added to the **Plan**, and before the team begins work on it. This is especially important if the **Plan** spans multiple projects. **To access and edit Plan properties:**\n\n1. Click the **Properties** button on the Plan detail pane toolbar (or Plan page toolbar if you are not viewing the Plan in the table). 2. Click the **Edit** button to put the Plan Properties form into edit mode. Note that this may not always be necessary, as many of the property fields can be edited in-place. Such fields provide visual feedback when you hover over them. ## Basic properties\n\nKey properties to focus on include:\n\n---\n\n3-20  \nSoftware Version Polarion 2410\n\nAdministrator and User Help  \nUnpublished work. © 2024 Siemens\n```\n\n<!-- End of Page 21 -->'),
 Document(metad

In [26]:
rag_chain.invoke({"question": "List the plan properties?"})["response"].content

'**Answer**  \nThe plan properties include basic properties and configuration properties.\n\n**Supporting details**  \n• Key properties to focus on include basic properties that should be reviewed and set before adding Work Items to the Plan (p. 21).  \n• Configuration properties that can be set include **Project Span**, which allows adding Work Items from multiple projects (p. 22).  \n• The types of Plans that can be created include **Release**, **Iteration**, **Milestone**, **Sprint**, **Kanban**, and **Omega Release** (p. 19).'

In [24]:
rag_chain.invoke({"question": "Explain the planning sidebar?"})["context"]

[Document(metadata={'_id': 'c872bc37-7c83-4c57-b3b3-9f4d3a1c0f3b', '_collection_name': 'polarion_admin_guide_semantic_chunks'}, page_content='```markdown\nPlanning sidebar\n================\n\nThis sidebar panel is available when viewing/editing a Plan in Table view, and when editing a Document. The **Planning sidebar** is opened automatically by the **Open in Table** action. ![Open in Table](#) **Actions** menu, or **Open in Table** widget in a **Plan**. Cannot be invoked from a **Document**.)\n\n> **Tip**  \n> The **Planning sidebar** can also be accessed by clicking the *Planned In* field in the *Work Item* table. The **Planning sidebar** enables you to load any open **Plan** in the table, or open its page. This can be useful when you are planning **Work Items** for a release or iteration. You can easily check any plan to see what **Work Items** have been added to it and what the workload is like, and to add or remove **Work Items** in different Plans until you have a release plan a

In [25]:
rag_chain.invoke({"question": "Explain the planning sidebar?"})["response"].content

'**Answer**  \nThe Planning sidebar is a tool in Polarion that allows users to manage Work Items within a selected Plan, facilitating the planning process for releases or iterations.\n\n**Supporting details**  \n• The Planning sidebar is available when viewing/editing a Plan in Table view and when editing a Document, and it opens automatically via the **Open in Table** action (p. 1).  \n• Users can access the Planning sidebar by clicking the *Planned In* field in the *Work Item* table (p. 1).  \n• It enables users to load any open Plan, view the workload, and add or remove Work Items from different Plans (p. 1).  \n• The sidebar displays the title and ID of the currently selected Plan, which is useful for referencing in reports or API calls (p. 2).  \n• It shows statistics about the selected Plan, including the count of Work Items planned for processing (p. 3).  \n• Changes to Plans are not saved until the **Save** button is clicked, which indicates how many Plans have pending changes 

### Integrating Evaluation

In [17]:
from langsmith import Client
from langsmith.evaluation import evaluate
from langsmith.evaluation.evaluator import RunEvaluator, EvaluationResult
from langchain_core.prompts import PromptTemplate
import re

# STEP 1: Define a custom Faithfulness + Relevance evaluator
class PolarionEvaluator(RunEvaluator):
    def __init__(self):
        from langchain_openai import ChatOpenAI
        self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
        self.prompt = PromptTemplate(
            input_variables=["question", "answer", "context"],
            template="""
Rate how faithful and relevant the answer is to the given context and question.

QUESTION:
{question}

ANSWER:
{answer}

CONTEXT:
{context}

Instructions:
- Is the answer supported by the context? (Faithfulness)
- Is it aligned with the question’s intent? (Relevance)
- Avoid penalizing format/citation issues.

Respond with:
Score: <0-100>
Reason step-by-step before giving the score.
"""
        )

    def evaluate_run(self, run, example=None):
        try:
            question = run.inputs["question"]
            answer = run.outputs["response"]
            context = run.outputs["context"]

            evaluation_input = self.prompt.format(question=question, answer=answer, context=context)
            result = self.llm.invoke(evaluation_input).content

            reasoning, score_str = result.rsplit("Score:", maxsplit=1)
            score = float(re.search(r"\d+", score_str).group()) / 100.0

            return EvaluationResult(
                key="polarion_eval_score",
                score=score,
                reasoning=reasoning.strip()
            )
        except Exception as e:
            return EvaluationResult(key="polarion_eval_score", score=None, comment=str(e))


In [20]:
client = Client()

dataset = client.create_dataset(
    "Polarion AI Asst Eval Dataset ",
    description="Eval dataset for Polarion ALM Assistant"
)

client.create_examples(
    inputs=[
        {"question": "How to convert home page from classic wiki format?"},
        {"question": "How to access user account page?"},
        {"question": "How to personalize navigation overview?"},
        {"question": "How to create a new favorite?"},
    ],
    dataset_id=dataset.id
)


{'example_ids': ['e7f9afd8-ffae-44b1-aaad-7325098e2f4d',
  '3f6529bc-1a9a-4b84-aeb0-7e87361c61fa',
  '27f2fa8d-1fa3-4c53-9a29-f12ccfd6384b',
  '819cb494-44d6-48cd-984a-bca710d506cf'],
 'count': 4}

In [21]:
def polarion_chain_fn(inputs):
    result = rag_chain.invoke(inputs)
    return {"response": result["response"].content, "context": result["context"]}

In [23]:
from langchain.smith import RunEvalConfig

eval_config = RunEvalConfig(
    custom_evaluators=[PolarionEvaluator()],
    project_name="Polarion AI Evaluation 1",
    project_metadata={"version": "1.0"}
)

client.run_on_dataset(
    dataset_name="Polarion AI Asst Eval Dataset",
    llm_or_chain_factory=polarion_chain_fn,
    evaluation=eval_config,
    verbose=True
)


View the evaluation results for project 'brief-middle-45' at:
https://smith.langchain.com/o/958971ef-386b-4c29-9daa-db720d891cdb/datasets/b0fb9688-f336-4d7f-b0b2-0af2c2d2bccb/compare?selectedSessions=cdffac3a-73b5-4ece-a6c2-6719260a4d07

View all tests for Dataset Polarion AI Asst Eval Dataset at:
https://smith.langchain.com/o/958971ef-386b-4c29-9daa-db720d891cdb/datasets/b0fb9688-f336-4d7f-b0b2-0af2c2d2bccb
[------------------------------------------------->] 4/4

{'project_name': 'brief-middle-45',
 'results': {'27f2fa8d-1fa3-4c53-9a29-f12ccfd6384b': {'input': {'question': 'How to access user account page?'},
   'feedback': [EvaluationResult(key='polarion_eval_score', score=1.0, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=None, target_run_id=None, extra=None)],
   'execution_time': 2.454442,
   'run_id': 'fdb7ddbe-0011-45e9-a44d-f71cbac5c869',
   'output': {'response': '**Answer**  \nTo access your user account page in Polarion, follow these steps:\n\n**Supporting details**  \n• Click the ![Navigation Icon](Navigation) on the top left of **Navigation**.  \n• Click ![My Account Icon](My Account) **My Account** (p. 3-6).',
    'context': [Document(metadata={'_id': '736b2b4a-14b8-4466-b327-ad45d056d6e3', '_collection_name': 'polarion_admin_guide_chunks_1'}, page_content='```markdown\n> **Note**\n> After you save any customization to the page, you can reset it to the default content any time by 